# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. 
Для этого объедините 2 массива в DataFrame. 
Обоснуйте выбор теста на корреляцию. 
Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [1]:
import pandas as pd #для работы с таблицами

В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [2]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

### Создание dataframe

In [3]:
# создание dataframe из исходных данныхS
df_SPb = pd.DataFrame(petersburg, columns = ['SPb_size'])
df_Mag = pd.DataFrame(magadan, columns = ['Magadan_size'])



### Объединение dataframe

In [4]:
df = df_Mag.join(df_SPb, rsuffix= 'right', how = 'left') 

In [5]:
display(df)

,Magadan_size,SPb_size
0,0.1033,0.0974
1,0.0915,0.1352
2,0.0781,0.0817
3,0.0685,0.1016
4,0.0677,0.0968
5,0.0697,0.1064
6,0.0764,0.1050
7,0.0689,NaN


In [6]:
#замена пропуска на среднее значение в столбце

df = df.fillna(df.mean())

## Проверка данных на нормальность

In [7]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [8]:
alpha = 0.05

## Тест Шапиро-Уилка

In [9]:
from scipy.stats import shapiro

In [10]:
_, p = shapiro(df)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.100
Данные распределены нормально


## Тест Д’Агостино

In [11]:
from scipy.stats import normaltest

In [12]:
_, p = normaltest(df)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.154
Данные распределены нормально


/home/alexandr/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


## Общий вывод
### Проведенные оба теста подтвердили, что данные в по размеру мидий распределены нормально.

## Определение корелляции

Выбран метод Пирсона, так как согласно тестам на нормальность, выполненным выше, данные распределены нормально. Соответственно, для такого распределения используется метод Пирсона.

In [13]:
from scipy.stats import pearsonr #загрузка модуля для расчета коэффициента корелляции методом Пирсона

In [14]:
#расчет корелляции
corr, p = pearsonr(df['SPb_size'], df['Magadan_size'])

In [15]:
print(corr, p)

0.24455753997284538 0.5594098062056255


In [16]:
from scipy.stats import f_oneway

In [17]:
_, p = f_oneway(df['SPb_size'], df['Magadan_size'])

### Проверка гипотезы влияния места на размер мидий

In [18]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в двух разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в двух разных местах.'

In [19]:
#проверка гипотезы о влиянии места на средний размер мидий
if p>alpha:
  print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.0027347063285687156 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в двух разных местах.


### Вывод
Нулевая гипотеза опровергнута об отсутствии влияния места на размер раковины мидий.
Соответсвенно, место влияет на размер раковины мидии.

# Независимый T-тест 

In [20]:
from scipy.stats import ttest_ind

Зависимая переменная (размер мидии) является количественной. Группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

In [21]:
H0 = 'Нет значимой разницы между размерами мидий в разных местах.'
Ha = 'Есть значимая разница между дразмерами мидий в разных местах.'

In [22]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['SPb_size'], df['Magadan_size'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(df)


*** Результаты независимого T-теста ***
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между дразмерами мидий в разных местах.


### Вывод по итогам проведенного т-теста.
Нулевая гипотеза опровергнута об отсутствии влияния места на размер раковины мидий.
Соответсвенно, место влияет на размер раковины мидии.